# Prediccion Click-through con Arboles de Decision

> **Proyecto a realizar** : Aplicar el estimador RandomForestClassifier para solucionar el problema de prediccion CTR sobre el conjunto de datos Avazu obteniendo un minimo de 80% de score AUC, sobre una validacion cruzada de 5 partes.

-  Utilizar busqueda en malla de 5 partes (GridSearchCV)
-  Buscar documentacion de la clase RandomForestClassifier para conocer los parametros que recibe el constructor de la misma.
-  Incluir en la busqueda en malla 4 de estos parametros (como mınimo) y 40 combinaciones de parametros y valores (tambien como mınimo).
- Graficar los atributos mas importantes determinados por el clasificador.






#### NOTA:

Este archivo fue hecho en equipo pero para manterner la privacidad de los integrantes se quitaron sus datos, a la par que se quitaron la mayoria de las casillas ya que eran demasiadas y solamente se dejaron las importantes, esto con el fin de comprobar su funcionalidad

## LIBRERIAS QUE VAMOS A USAR

In [1]:
from sklearn.linear_model import Lasso
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV
import copy
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import cross_val_score


---
## Funciones que vamos a utilizar para relizar el proyecto

In [2]:
def read_ad_click_data(filename, n, offset=0):
    """ Read training instances from the Click-Through Rate Prediction dataset
       (https://www.kaggle.com/c/avazu-ctr-prediction)
    Args:
        filename - Name of file to read instances from
        n - Number of instances to read
        offset - Instance number to begin reading from
    Returns:
        X_dict, y - Features of the read instances (as a dictionary) and corresponding labels (as a list)
        
    Note:
        We exclude features 'id', 'hour', 'device_id', and 'device_ip' from the dataset in order to avoid
        a huge number of features coming from the OHE of these categorical attributes
    """
    X_dict, y = [], []
    
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for i in range(offset):
            next(reader)
            
        i = 0
        for row in reader:
            i += 1
            y.append(int(row['click']))
            del row['click'], row['id'], row['hour'], row['device_id'], row['device_ip']
            X_dict.append(dict(row))
            
            if i >= n:
                break
                
    return X_dict, y

---
## Leemos los datos de los archivos CSV  

In [3]:
n_max =10000 # can be up to 100,000 instances (but requieres A LOT of memory to process)5
X_dict_train, y_train = read_ad_click_data('train_ctr.csv', n_max)
X_dict_test, y_test = read_ad_click_data('test_ctr.csv', n_max)

---
## Declaramos nuestras variables de entrenamiento y de prueba aplicando el OHE

Usamos el OHE debido a que hay algunas columnas que no son numericas, a lo cual nos impide trabajar con ellas a lo menos que la pasemos a formato numerico

In [4]:
dict_one_hot_encoder = DictVectorizer(sparse=False)
X_train = dict_one_hot_encoder.fit_transform(X_dict_train)
X_test = dict_one_hot_encoder.transform(X_dict_test)

---
## Visualizamos los datos

Si todo salio bien durante esta parte del proyecto, entonces tendriamos en nuestras variables *$ X-dict-train,X-dict-test $* 30,000 datos y 19 columnas. Y por la parte de  *$ X-train,X-test $* deberiamos tener mucho mas columnas 19 debiado al OHE

In [5]:
print("Num_Datos:",len(X_dict_train), "Num_Columnas:",len(X_dict_train[0]))
print("---- APLICACION DEL OHE ----")
print("Num_colmnas despues de aplicar OHE:",len(X_test[0]))

Num_Datos: 10000 Num_Columnas: 19
---- APLICACION DEL OHE ----
Num_colmnas despues de aplicar OHE: 2820


---
**$$  ANALISIS {\thinspace} DE {\thinspace} LOS {\thinspace} DATOS  $$**

> Antes de continuar debemos de hacer un pequeño analisis sobre que tenemos que hacer y que podemos hacer ahora para evitar "trabajar de mas". Lo que nos queremos referir con esto es, que casi siempre al momento de que alguien aplica **OHE** y quiere obtener una prediccion o lo que sea, tiende a generarse *$ruido$*. A lo que hemos optado por aplicar **Lasso** ya que este metodo nos permite saber cuales columnas de nuestros conjuntos son "inutiles" o no tiene sentido tomarlas en cuenta.
 

 **Recordatorio de Lasso:**  Para poder usar Lasso haremos uso de la libreria Sklearn que ya nos proporciona dicho metodo, mas sin embargo cabe recordar que Lasso tiene un parametro llamado ** alpha ** dicho parametro es necesario encontrar el optimo para evitar que Lasso no sea ni muy estricto ni tampoco que sea muy ligero al momento de evaluar las columnas.
 
**Aplicacion de  Lasso:** Entonces para poder aplicar Lasso como ya habiamos mencionado previamente, vamos a hacer uso de la libreria Sklearn, pero aparte de hacer uso de Lasso tambien vamos a necesitar de la **Validacion Cruzada** o tambien conocida por su nombre en ingles **Cross Val Socre**, el codigo de ambos se escribe de la siguiente manera:



``` python

# Para usar Lasso

from sklearn.linear_model import Lasso

>>> varaible_lasso = Lasso(alpha=n) # <- Aqui decimos que vamos a usar lasso con una alpha=n donde n es un 'float'
>>> variable_lasso.fit(X_train,y_train) # <- Aqui le decimos que aplique lasso a nuestros dos conjuntos

# Para usar Cross Val Score

from sklearn.model_selection import cross_val_score

>>> variable_CVS = cross_val_score(metodo, X, y, scoring="neg_mean_squared_error", cv=5)

```




En el cual vamos a estar moviento los valores de **alpha** para encontrar el menor error posible segun Lasso, pero el "problema" que nos puede pasar ahora es que, nuestras varibale **X_train** al tener 30,000 datos  y mas de 4000 columnas, este proceso le va a llevar mucho tiempo a Lasso (en nuestro caso se tardaba 10min aproximadamente, cada que cambiamos el parametro **alpha**).  Asi que se les recomienda que apartir de aqui en adelante se tenga un buen equipo computacional para poder ejecutuar las lineas de codigos siguientes. (En nuestro caso a los mas que llego a usar fue 8gb de ram debido a que pusimos muchos datos y muchas **alphas**)

Lo que vamos a hacer ahora sera hacer pruebas con diferentes **alphas** y al final mostraremos una tabla para comparar resultados

### $$ Para {\thinspace} alpha {\thinspace} = {\thinspace} 0.1 $$

In [6]:
lasso_reg = Lasso(alpha=0.1)
scores = cross_val_score(lasso_reg, X_train, y_train,scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt( - scores)
print(np.mean(rmse_scores))

0.376152776744873


### $$ Para {\thinspace} alpha {\thinspace} = {\thinspace} 0.01 $$

In [7]:
lasso_reg = Lasso(alpha=0.01)
scores = cross_val_score(lasso_reg, X_train, y_train,scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt( - scores)
print(np.mean(rmse_scores))

0.3691060783955664


### $$ Para {\thinspace} alpha {\thinspace} = {\thinspace} 0.001 $$

In [8]:
lasso_reg = Lasso(alpha=0.001)
scores = cross_val_score(lasso_reg, X_train, y_train,scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt( - scores)
print(np.mean(rmse_scores))

0.36196876020902113


### $$ Para {\thinspace} alpha {\thinspace} = {\thinspace} 0.0001 $$

In [9]:
lasso_reg = Lasso(alpha=0.0001)
scores = cross_val_score(lasso_reg, X_train, y_train,scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt( - scores)
print(np.mean(rmse_scores))

0.36196661886901593


## $$ TABLA{\thinspace}{\thinspace}COMPARATIVA  $$

Despues de haber aplicado Lasso con multplies **alphas** diferentes, es momento de saber cual es el mejor **alpha** para nuestro proyecto, para eso usaremos la tabal de abajo, donde nos dice el **alpha** que hayamos usado, la cantidad de datos (esto en el debido caso de querer trabajar en diferentes conjuntos) y finalmente su error o tambien conocido como $RMSE$. Donde al analizar esta tabla llegamos a la conclusion clara de que el mejor parametro que podemos usar es **alpha = 0.0001**


|  alpha    |  Cantidad Datos     | RMSE  |
|-------   |:--------------:|-------------------:|
|0.0001|10,000|0.3610409598229232|
|0.001|10,000|0.3619687602090211|
|0.01|10,000  |0.3691060783955663|
|0.1|10,000|0.376152776744873|




---
## $$ ACTUALIZACION{\thinspace}DE{\thinspace}LOS{\thinspace}DATOS $$

> Una vez que encontramos el mejor alpha para Lasso, ahora toca quitar las columnas que dijo Lasso que no tienen sentido usar y despues actualizar los datos. Para ello usaremos algo que tambien ya lo tiene implementado Sklearn y es ** coef_ **, la cual nos trae el valor que le dio Lasso a cada columna.


In [10]:
# Primero aplicamos el mejor alpha a lasso a los conjuntos X_train,y_train
lasso_r = Lasso(alpha=0.0001)
lasso_r.fit(X_train,y_train)

Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

#### ---Una vez aplicado Lasso a nuestros conjuntos de entrenamiento, tenemos que saber cuales columnas si nos interesan para nuestro proyecto, para ello haremos lo siguiente---

In [11]:
index_lasso = lasso_r.coef_
sirve = [] # Guardaremos las columnas que si nos interesan
for i,j in enumerate(index_lasso): #Recorremos toda la lista
    if j != 0:
        sirve.append(i)

In [12]:
parameters = {'n_estimators': [130],'max_depth':[37,40], 'min_samples_split': [313,314,315], 'max_leaf_nodes': [141,145,None]}
random_forest = RandomForestClassifier(criterion='gini',n_jobs=-1)

In [13]:
grid_search = GridSearchCV(random_forest, parameters, n_jobs=-1, cv=5, scoring='roc_auc', iid=True, verbose=2)
grid_search.fit(X_train[:,sirve], y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  3.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [130], 'max_depth': [37, 40], 'min_samples_split': [313, 314, 315], 'max_leaf_nodes': [141, 145, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [14]:
grid_search.best_params_

{'max_depth': 37,
 'max_leaf_nodes': 141,
 'min_samples_split': 314,
 'n_estimators': 130}

In [15]:
randomforest_best = grid_search.best_estimator_
pos_prob = randomforest_best.predict_proba(X_test[:,sirve])[:, 1] # column 0 - Negative class; column 1 - Positive class

In [16]:
from sklearn.metrics import roc_auc_score

print('The ROC AUC on testing set is: {0:.3f}'.format(roc_auc_score(y_test, pos_prob)))

The ROC AUC on testing set is: 0.722


---